In [1]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [2]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
import os
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model

load_dotenv()

# 试试qwen3-max的结构化输出效果，模型qwen-max的效果不行
model = init_chat_model("openai:qwen3-max",
                        temperature=0.5,
                        timeout=10,
                        max_tokens=1000,
                        base_url=os.getenv('BASE_URL'),
                        api_key=os.getenv('API_KEY'))

In [4]:
from dataclasses import dataclass

# 在结构化输出上，deepseek的表现要比qwen-max强。使用qwen3-max模型的效果也ok，支持结构化输出。

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

In [5]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [7]:
from langchain.agents.structured_output import ToolStrategy
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)
# agent = create_agent(
#     model = "deepseek-chat",
#     tools=[get_user_location, get_weather_for_location],
#     system_prompt=SYSTEM_PROMPT,
#     context_schema=Context,
#     response_format=ToolStrategy(ResponseFormat),
#     checkpointer=checkpointer
# )

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])

response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config = config,
    context=Context(user_id="1")
)

print(response['structured_response'])

ResponseFormat(punny_response='Florida’s shining bright again—looks like the sun’s *on a roll*! Don’t forget your sunscreen unless you want to be a crispy critter!', weather_conditions='sunny')
ResponseFormat(punny_response="You're welcome! Keep riding those sunny vibes—just don’t *solar* your responsibilities! 😎", weather_conditions='sunny')
